In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import functions
import plotly.express as px
import numpy as np

# Download the data

In [32]:
year=2021

In [3]:
functions.download_demand(year)
# download generation data
functions.download_generation(year)
# download emissions data
functions.download_emissions(year)

# Load the data
Interpolate missing data and resample to the hourly level

In [30]:
generation = functions.load_hourly_data(year, "generation")
emissions = functions.load_hourly_data(year, "emissions")

# Calculate emission factors for system power
 - CO2 data is in units of mTCO2/hr
 - generation data is in units of MW
 - to calculate an emissions factor (mTCO2/MWh), divide CO2 rate by generation


In [31]:
# only keep columns of fuels to keep
system_emissions = emissions.filter(regex="natural_gas|imports|coal").copy()
system_generation = generation.filter(regex="natural_gas|imports|coal").copy()

# imports are reported as net imports, so we want to exlcude hours when CAISO was exporting
system_generation.loc[system_generation['imports_mw'] < 0,'imports_mw'] = 0

# we also want to remove any negative generation and emission values
system_generation[system_generation < 0] = np.NaN
system_emissions[emissions < 0] = np.NaN

# sum all columns
system_emissions = system_emissions.sum(axis=1, skipna=False)
system_generation = system_generation.sum(axis=1, skipna=False)

hourly_ef = system_emissions / system_generation

hourly_ef = hourly_ef.rename("system_ef_co2_tonnes_per_mwh")

hourly_ef.to_csv(f'data/processed/hourly_ef_{year}.csv', index=False)
